# RNN 4D - no collapsing
# V2

In [1]:
import nibabel as nib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import numpy as np
import glob
import nibabel as nib
import os
import matplotlib.pyplot as plt
import scipy.ndimage
import random
from tensorflow.keras.layers import Dropout, Dense, Reshape, Flatten, Conv3D, Conv3DTranspose, LeakyReLU, Input, Embedding, multiply, Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization, Bidirectional, AdditiveAttention, LayerNormalization
from functools import partial
from tensorflow.keras import models, layers

In [2]:
#pip install nibabel

In [3]:
#pip install numpy==1.24.0

In [4]:
full_schizophrenia_ids = [
    'A00009280', 'A00028806', 'A00023132', 'A00014804', 'A00016859', 'A00021598', 'A00001181', 'A00023158',
    'A00024568', 'A00028405', 'A00001251', 'A00000456', 'A00015648', 'A00002405', 'A00027391', 'A00016720',
    'A00018434', 'A00016197', 'A00027119', 'A00006754', 'A00009656', 'A00038441', 'A00012767', 'A00034273',
    'A00028404', 'A00035485', 'A00024684', 'A00018979', 'A00027537', 'A00004507', 'A00001452', 'A00023246',
    'A00027410', 'A00014719', 'A00024510', 'A00000368', 'A00019293', 'A00014830', 'A00015201', 'A00018403',
    'A00037854', 'A00024198', 'A00001243', 'A00014590', 'A00002337', 'A00024953', 'A00037224', 'A00027616',
    'A00001856', 'A00037619', 'A00024228', 'A00038624', 'A00037034', 'A00037649', 'A00022500', 'A00013216',
    'A00020787', 'A00028410', 'A00002480', 'A00028303', 'A00020602', 'A00024959', 'A00018598', 'A00014636',
    'A00019349', 'A00017147', 'A00023590', 'A00023750', 'A00031597', 'A00015518', 'A00018317', 'A00016723',
    'A00021591', 'A00023243', 'A00017943', 'A00023366', 'A00014607', 'A00020414', 'A00035003', 'A00028805',
    'A00029486', 'A00000541', 'A00028408', 'A00000909', 'A00031186', 'A00000838' ]

# schizohrenia_id that satisfy t>90, 59 in total
met_requirement_schizophrenia_ids = [
    'A00000368', 'A00000456', 'A00000541', 'A00000838', 'A00001251', 'A00001452', 'A00004507',
    'A00006754', 'A00009280', 'A00012767', 'A00013216', 'A00014607', 'A00014719', 'A00014804',
    'A00014830', 'A00015201', 'A00015648', 'A00016197', 'A00016720', 'A00016723', 'A00017147',
    'A00018317', 'A00018403', 'A00018434', 'A00018979', 'A00019293', 'A00020414', 'A00020602', 
    'A00020787', 'A00021591', 'A00021598', 'A00023158', 'A00023246', 'A00023590', 'A00023750', 
    'A00024198', 'A00024228', 'A00024568', 'A00024684', 'A00024953', 'A00024959', 'A00027410', 
    'A00027537', 'A00028303', 'A00028404', 'A00028408', 'A00028805', 'A00028806', 'A00031186', 
    'A00031597', 'A00034273', 'A00035003', 'A00035485', 'A00037034', 'A00037224', 'A00037619', 
    'A00037649', 'A00038441', 'A00038624']

full_control_ids = [
    'A00007409', 'A00013140', 'A00021145', 'A00036049', 'A00022810', 'A00002198', 'A00020895', 'A00004667',
    'A00015826', 'A00023120', 'A00022837', 'A00010684', 'A00009946', 'A00037318', 'A00033214', 'A00022490',
    'A00023848', 'A00029452', 'A00037564', 'A00036555', 'A00023095', 'A00022729', 'A00024955', 'A00024160',
    'A00011725', 'A00027487', 'A00024446', 'A00014898', 'A00015759', 'A00028409', 'A00017294', 'A00014522',
    'A00012995', 'A00031764', 'A00025969', 'A00033147', 'A00018553', 'A00023143', 'A00036916', 'A00028052',
    'A00023337', 'A00023730', 'A00020805', 'A00020984', 'A00000300', 'A00010150', 'A00024932', 'A00035537',
    'A00022509', 'A00028406', 'A00004087', 'A00035751', 'A00023800', 'A00027787', 'A00022687', 'A00023866',
    'A00021085', 'A00022619', 'A00036897', 'A00019888', 'A00021058', 'A00022835', 'A00037495', 'A00026945',
    'A00018716', 'A00026907', 'A00023330', 'A00016199', 'A00037238', 'A00023131', 'A00014120', 'A00021072',
    'A00037665', 'A00022400', 'A00003150', 'A00024372', 'A00021081', 'A00022592', 'A00022653', 'A00013816',
    'A00014839', 'A00031478', 'A00014225', 'A00013363', 'A00037007', 'A00020968', 'A00024301', 'A00024820',
    'A00035469', 'A00029226', 'A00022915', 'A00022773', 'A00024663', 'A00036844', 'A00009207', 'A00024535',
    'A00022727', 'A00011265', 'A00024546'
]

 # 82 controls that met requirement
met_requirement_control_ids = [
    'A00000300', 'A00002198', 'A00003150', 'A00004087', 'A00007409', 'A00010684', 'A00011265', 'A00011725',
    'A00012995', 'A00013140', 'A00013816', 'A00014839', 'A00014898', 'A00015759', 'A00015826', 'A00018553',
    'A00018716', 'A00019888', 'A00020805', 'A00020895', 'A00020968', 'A00020984', 'A00021058', 'A00021072',
    'A00021081', 'A00021085', 'A00022400', 'A00022490', 'A00022509', 'A00022592', 'A00022619', 'A00022653',
    'A00022687', 'A00022727', 'A00022729', 'A00022773', 'A00022810', 'A00022835', 'A00022837', 'A00022915',
    'A00023095', 'A00023120', 'A00023131', 'A00023143', 'A00023330', 'A00023337', 'A00023730', 'A00023800',
    'A00023848', 'A00023866', 'A00024160', 'A00024301', 'A00024372', 'A00024446', 'A00024535', 'A00024546', 
    'A00024663', 'A00024820', 'A00024932', 'A00024955', 'A00025969', 'A00026945', 'A00027487', 'A00027787', 
    'A00028052', 'A00028406', 'A00028409', 'A00029226', 'A00029452', 'A00031478', 'A00031764', 'A00033214', 
    'A00035751', 'A00036049', 'A00036555', 'A00036844', 'A00037007', 'A00037238', 'A00037318', 'A00037495', 
    'A00037564', 'A00037665'
]

In [5]:
# import nibabel as nib
# import os
# import shutil

# # Directory containing your .nii.gz files
# directory_path = '4D/'
# file_pattern = '*.nii.gz'  # Adjust as needed for your file pattern

# # Directory to move corrupt files, create if doesn't exist
# corrupt_files_dir = os.path.join(directory_path, 'corrupt_files')
# os.makedirs(corrupt_files_dir, exist_ok=True)

# # List to store paths of corrupt files
# corrupt_files = []

# # Iterate over all files in the directory
# for root, _, files in os.walk(directory_path):
#     for file in files:
#         if file.endswith('.nii.gz'):
#             file_path = os.path.join(root, file)
#             try:
#                 # Attempt to load the file
#                 t1_img = nib.load(file_path)
#                 # Attempt to read the data to ensure it's not corrupt
#                 t1_data = t1_img.get_fdata()

#             except (EOFError, OSError, nib.filebasedimages.ImageFileError) as e:
#                 # Log the corrupt file and the error message
#                 print(f"Corrupt file detected: {file_path} | Error: {e}")
#                 corrupt_files.append(file_path)

#                 # Optionally, move the corrupt file to a separate directory
#                 shutil.move(file_path, os.path.join(corrupt_files_dir, file))
#                 continue

# # Output the list of corrupt files
# if corrupt_files:
#     print(f"\nTotal corrupt files found: {len(corrupt_files)}")
#     for corrupt_file in corrupt_files:
#         print(corrupt_file)
# else:
#     print("No corrupt files found.")


In [6]:
# GAN Training Data Selection
gan_train_ids_schiz = random.sample(met_requirement_schizophrenia_ids, 50)
gan_test_ids_schiz = [id for id in met_requirement_schizophrenia_ids if id not in gan_train_ids_schiz]

gan_train_ids_control = random.sample(met_requirement_control_ids, 50)
gan_test_ids_control = [id for id in met_requirement_control_ids if id not in gan_train_ids_control]
gan_test_ids_control = random.sample(gan_test_ids_control,9)

''' data training for classifier '''
''' just use the same train set as GAN above '''

# Classifier Test Data Selection
classifier_test_ids = gan_test_ids_schiz + gan_test_ids_control

''' File loading '''
# Specify the directory and file pattern
directory_path = '4D/'
file_pattern = 'A*_????_func_FL_FD_RPI_DSP_MCF_SS_SM_Nui_CS_InStandard.nii.gz'

# Construct the full path pattern
path_pattern = f'{directory_path}/{file_pattern}'

# Use glob to find all matching files
matching_files = glob.glob(path_pattern)

''' File loading for GAN Training and classifer '''
''' But this time we have 2 separate GANs, 1 train on schizoprenia and 1 train on control'''

#classifier_image_data = []
#classifier_labels = []  # 1 for schizophrenia, 0 for non-schizophrenia
gan_image_data_schiz = []
gan_image_data_control = []

for file_path in matching_files:
    filename = os.path.basename(file_path)
    file_id = filename.split('_')[0]
    
    if file_id in gan_train_ids_schiz:
        t1_img = nib.load(file_path)
        t1_data = t1_img.get_fdata()
        

        if t1_data.shape[3] < 90:
            continue

        #t1_data = np.sum(t1_data, axis=1)
        #print('shape of image: ', t1_data.shape)
        gan_image_data_schiz.append(t1_data)

    if file_id in gan_train_ids_control:
        t1_img = nib.load(file_path)
        t1_data = t1_img.get_fdata()

        if t1_data.shape[3] < 90:
            continue

        #t1_data = np.sum(t1_data, axis=1)
        gan_image_data_control.append(t1_data)


print(f"Total GAN control loaded: {len(gan_image_data_control)}")
print(f"Total GAN schiz loaded: {len(gan_image_data_schiz)}")



'''Determine the maximum time-dimension size '''
max_z_size_schiz = max(img.shape[3] for img in gan_image_data_schiz)
max_z_size_control = max(img.shape[3] for img in gan_image_data_control)
max_t_size = max(max_z_size_schiz,max_z_size_control)


# Normalize and pad the data
def normalize_and_pad(data, max_t):
    normalized = (data - np.min(data)) / (np.max(data) - np.min(data)) * 2 - 1
    padded = np.pad(normalized, ((0, 0), (0, 0), (0, 0), (0, max_t - data.shape[3])), mode='constant')
    return padded

padded_data_schiz = [normalize_and_pad(img, max_t_size) for img in gan_image_data_schiz]
padded_data_control = [normalize_and_pad(img, max_t_size) for img in gan_image_data_control]

padded_data_array_schiz = padded_data_schiz
padded_data_array_control = padded_data_control
print("shape after normalization and padding", padded_data_array_control[0].shape)

Total GAN control loaded: 50
Total GAN schiz loaded: 50
shape after normalization and padding (91, 109, 91, 146)


In [8]:
# This approach is too naive

# batch_size = 10
# # Flatten each 3D volume for each time step
# def flatten_3d_data(data):
#     return np.reshape(data, (data.shape[3], data.shape[0] * data.shape[1] * data.shape[2]))  # (t, x*y*z)

# # Prepare the data for the LSTM model
# flattened_data_schiz = [flatten_3d_data(img) for img in padded_data_array_schiz]
# flattened_data_control = [flatten_3d_data(img) for img in padded_data_array_control]
# print("shape after flattening", flattened_data_schiz[0].shape)

# # Convert to numpy arrays
# train_images_schiz = np.array(flattened_data_schiz)
# train_images_control = np.array(flattened_data_control)


# # Create labels
# labels_schiz = np.ones(len(padded_data_array_schiz))
# labels_control = np.zeros(len(padded_data_array_control))

# # Combine the datasets
# train_images = np.concatenate((train_images_schiz, train_images_control), axis=0)
# train_labels = np.concatenate((labels_schiz, labels_control), axis=0)

# # Shuffle the dataset
# indices = np.arange(train_images.shape[0])
# np.random.shuffle(indices)
# train_images = train_images[indices]
# train_labels = train_labels[indices]

# # Create TensorFlow dataset
# train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
# train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

In [9]:
import numpy as np
import tensorflow as tf

# Batch size
batch_size = 10

# Function to flatten 3D volumes for each time step (t, x*y*z)
def flatten_3d_data(data):
    return np.reshape(data, (data.shape[3], data.shape[0] * data.shape[1] * data.shape[2]))  # (t, x*y*z)

# Flatten each 3D volume for each time step
flattened_data_schiz = [flatten_3d_data(img) for img in padded_data_array_schiz]
flattened_data_control = [flatten_3d_data(img) for img in padded_data_array_control]
print("Shape after flattening:", flattened_data_schiz[0].shape)  # Check the shape after flattening

# Create labels
labels_schiz = np.ones(len(flattened_data_schiz))
labels_control = np.zeros(len(flattened_data_control))

# Combine images and labels
train_images = flattened_data_schiz + flattened_data_control
train_labels = np.concatenate((labels_schiz, labels_control), axis=0)

# Shuffle indices
indices = np.arange(len(train_images))
np.random.shuffle(indices)

# Shuffle data based on indices
train_images = [train_images[i] for i in indices]
train_labels = train_labels[indices]

# Define a generator function to yield data batches
def data_generator(images, labels, batch_size):
    for i in range(0, len(images), batch_size):
        batch_images = images[i:i + batch_size]
        batch_labels = labels[i:i + batch_size]
        yield np.array(batch_images), np.array(batch_labels)

# Create TensorFlow Dataset from the generator
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(train_images, train_labels, batch_size),
    output_signature=(
        tf.TensorSpec(shape=(None, max_t_size, 91 * 109 * 91), dtype=tf.float32),  # Adjust shape if needed
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
)

# Prefetch for performance improvement
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Debug: Test the generator
for images, labels in train_dataset.take(1):
    print(f"Batch image shape: {images.shape}")
    print(f"Batch labels shape: {labels.shape}")


Shape after flattening: (146, 902629)
Batch image shape: (10, 146, 902629)
Batch labels shape: (10,)


In [10]:
def build_rnn_model():
    
    # Define input shape: (time_steps, flattened_features)
    time_steps = max_t_size  # Number of time points
    features_per_volume = 91 * 109 * 91  # Flattened size of each 3D volume

    # Input shape for LSTM
    input_shape = (time_steps, features_per_volume)

    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))

    # LSTM layers to process the sequence
    model.add(layers.LSTM(128, return_sequences=True))
    model.add(layers.LSTM(64))
    #model.add(layers.Dropout(0.5))

    # Output layer for binary classification
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model
    
rnn_model = build_rnn_model()
rnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 146, 128)          462212096 
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 462261569 (1.72 GB)
Trainable params: 462261569 (1.72 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')

# Define number of epochs
epochs = 15

# Training loop
for epoch in range(epochs):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    for images, labels in train_dataset:
        #images = tf.transpose(images, perm=[0, 4, 1, 2, 3, 5])  # (batch, t, x, y, z, channels)
        
        with tf.GradientTape() as tape:
            predictions = rnn_model(images, training=True)
            loss = tf.keras.losses.binary_crossentropy(labels, tf.squeeze(predictions, axis=1))
        
        gradients = tape.gradient(loss, rnn_model.trainable_variables)
        rnn_model.optimizer.apply_gradients(zip(gradients, rnn_model.trainable_variables))
        
        # Update the metrics
        train_loss.update_state(loss)
        train_accuracy.update_state(labels, tf.squeeze(predictions, axis=1))
    
    # Print the loss and accuracy for the current epoch
    print(f"Epoch {epoch + 1}, Loss: {train_loss.result().numpy()}, Training Accuracy: {train_accuracy.result().numpy()}")


Epoch 1, Loss: 0.9121697545051575, Training Accuracy: 0.5400000214576721
Epoch 2, Loss: 0.7190192341804504, Training Accuracy: 0.47999998927116394
Epoch 3, Loss: 0.6927342414855957, Training Accuracy: 0.5099999904632568
Epoch 4, Loss: 0.6929518580436707, Training Accuracy: 0.5400000214576721
Epoch 5, Loss: 0.6923525333404541, Training Accuracy: 0.5400000214576721
Epoch 6, Loss: 0.6914445161819458, Training Accuracy: 0.550000011920929
Epoch 7, Loss: 0.6916589736938477, Training Accuracy: 0.5400000214576721
Epoch 8, Loss: 0.6912298202514648, Training Accuracy: 0.5400000214576721
Epoch 9, Loss: 0.6909211874008179, Training Accuracy: 0.5400000214576721
Epoch 10, Loss: 0.6912838220596313, Training Accuracy: 0.5400000214576721
Epoch 11, Loss: 0.6909537315368652, Training Accuracy: 0.5400000214576721
Epoch 12, Loss: 0.6914007067680359, Training Accuracy: 0.550000011920929
Epoch 13, Loss: 0.6913293600082397, Training Accuracy: 0.550000011920929
Epoch 14, Loss: 0.6909869909286499, Training Accu

In [12]:
def resize_image(image, new_shape):
    factors = (
        new_shape[0] / image.shape[0],
        new_shape[1] / image.shape[1],
        new_shape[2] / image.shape[2]
    )
    return scipy.ndimage.zoom(image, factors, order=1)  # order=1 is bilinear interpolation


test_image_data = []
test_labels = []

test_ids = classifier_test_ids  # List of IDs to filter test data

for file_path in matching_files:
    filename = os.path.basename(file_path)
    file_id = filename.split('_')[0]
    
    if file_id in test_ids:
        t1_img = nib.load(file_path)
        t1_data = t1_img.get_fdata()

        if t1_data.shape[3] < 90:
            continue
        

        # Normalize the processed image
        processed_image_normalized = (t1_data - np.min(t1_data)) / (np.max(t1_data) - np.min(t1_data)) * 2 - 1

        
        # Pad or truncate the time dimension to match the expected size (max_t_size)
        current_t_size = processed_image_normalized.shape[3]
        
        if current_t_size < max_t_size:
            # Pad to max_t_size time steps if the current time dimension is smaller
            pad_size = max_t_size - current_t_size
            #pad_size = max_t_size
            processed_image_padded = np.pad(processed_image_normalized, ((0, 0), (0, 0), (0, 0), (0, pad_size)), mode='constant')
        elif current_t_size > max_t_size:
            # Truncate to max_t_size time steps if the current time dimension is larger
            processed_image_padded = processed_image_normalized[:, :, :, :max_t_size]
        else:
            processed_image_padded = processed_image_normalized  # No padding needed

        # Check final shape before flattening
        #print(f"Processed image padded shape: {processed_image_padded.shape}")

        # Flatten each 3D volume (91, 109, 91) for each time step (t)
        flattened_shape = (processed_image_padded.shape[3], processed_image_padded.shape[0] * processed_image_padded.shape[1] * processed_image_padded.shape[2])
        processed_image_flattened = np.reshape(processed_image_padded, flattened_shape)  # (t, x*y*z)

        # Verify the shape after flattening
        #print(f"Flattened image shape: {processed_image_flattened.shape}")

        test_image_data.append(processed_image_flattened)
        
        label = 1 if file_id in met_requirement_schizophrenia_ids else 0
        test_labels.append(label)

# Convert to numpy arrays for easier handling in TensorFlow
test_images_array = np.array(test_image_data)
test_labels_array = np.array(test_labels)

# Check the final shape of test images
#print("Final shape of test_images_array:", test_images_array.shape)
#print("Final shape of test_labels_array:", test_labels_array.shape)

#batch_size=1

# Create a TensorFlow dataset from the numpy arrays
test_dataset = tf.data.Dataset.from_tensor_slices((test_images_array, test_labels_array)).batch(batch_size)


In [13]:
# Evaluate the model
loss, accuracy = rnn_model.evaluate(test_dataset)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

2/2 [==============================] - 45s 3s/step - loss: 0.6855 - accuracy: 0.5556
Test loss: 0.685500979423523, Test accuracy: 0.5555555820465088


In [14]:
# Predict the probabilities
predictions = rnn_model.predict(test_dataset)
# Convert probabilities to class labels
predicted_labels = (predictions > 0.5).astype(int)

# Assuming test_labels_array contains your actual labels
actual_labels = test_labels_array

# Now you might want to compare these predicted_labels with the actual labels (test_labels)
# to compute the confusion matrix, classification report, etc.
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(actual_labels, predicted_labels.flatten()))
print(classification_report(actual_labels, predicted_labels.flatten()))

2/2 [==============================] - 18s 8s/step
[[7 2]
 [6 3]]
              precision    recall  f1-score   support

           0       0.54      0.78      0.64         9
           1       0.60      0.33      0.43         9

    accuracy                           0.56        18
   macro avg       0.57      0.56      0.53        18
weighted avg       0.57      0.56      0.53        18



In [15]:
for actual, predicted in zip(actual_labels, predicted_labels):
    print(f'Actual: {actual}, Predicted: {predicted}')

Actual: 1, Predicted: [1]
Actual: 0, Predicted: [0]
Actual: 0, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 0, Predicted: [0]
Actual: 0, Predicted: [0]
Actual: 0, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 0, Predicted: [0]
Actual: 0, Predicted: [1]
Actual: 0, Predicted: [0]
Actual: 0, Predicted: [1]
Actual: 1, Predicted: [1]
